# Загрузка библиотек

In [1]:
import pandas as pd
import re
import seaborn as sns
from tqdm import tqdm_notebook

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import time

from tqdm.notebook import tqdm

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Загрузка данных для предсказания 

In [2]:
df_test = pd.read_csv('test_dataset_test.csv')

In [3]:
df_test.head()

,id,Текст Сообщения,Тематика,Ответственное лицо
0,843,<p>Здравствуйте. На улице Мира &nbsp;было заме...,Неработающее наружное освещение,Администрация Курчатовского района
1,1422,<p>Уже вторую неделю не горит уличное освещени...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
2,2782,Не работает освещение во дворе дома 11а по Эне...,Неработающее наружное освещение,Комитет жилищно-коммунального хозяйства города...
3,2704,После покоса сорной растительности на газоне м...,Неудовлетворительная уборка улиц и тротуаров,Администрация Центрального округа города Курска
4,1,<p>Прошу принять меры к водителю маршрута 263:...,Неудовлетворительный внешний вид (поведение) в...,Администрация города Курска


In [4]:
submission = df_test[["id", "Текст Сообщения"]]

In [5]:
submission.columns = ['id', 'text']

In [6]:
def preprocessing_text(text):
    text = text.lower()
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)
    text = re.sub('\s+', ' ', text)
    return text

# Загрузка модели

In [7]:
device = "cuda" if torch.cuda.is_available else "cpu"

In [8]:
model_name1 = 'DeepPavlov/rubert-base-cased'

In [9]:
model_1 = AutoModelForSequenceClassification.from_pretrained(model_name1)
model_2 = AutoModelForSequenceClassification.from_pretrained(model_name1)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [10]:
model_1.classifier = nn.Linear(in_features=768, out_features=17, bias=True)
model_2.classifier = nn.Linear(in_features=768, out_features=17, bias=True)
model_1.load_state_dict(torch.load('best-val-model_bert.pt'))
model_2.load_state_dict(torch.load('best-val-model_bert_full.pt'))
model_1 = model_1.to(device)
model_2 = model_2.to(device)
model_1.eval()
model_2.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name1)
max_length = 512

# Тестовое предсказание

In [12]:
test_text = submission.iloc[0]['text']

In [13]:
test_text

'<p>Здравствуйте. На улице Мира &nbsp;было заменено наружное освещение, а именно заменены лампы на &nbsp;энергосберегающие лампы. На протяжении нескольких месяцев освещение улицы отсутствует. Последний раз когда улица была освещена это зима приблизительно до 23:00 да и то не каждый день. А ведь люди работают в 12 часовую смену и многие возвращаются очень поздно. Данная проблема на всех улицах поселка.&nbsp;</p>'

In [16]:
# проверка решения
with torch.no_grad():
    tokenizer_output1 = tokenizer.encode_plus(test_text,
                                          max_length=max_length,
                                          padding="max_length",
                                          return_tensors="pt"
                                         )
    token = tokenizer_output1['input_ids'][:max_length].to(device)
    mask = tokenizer_output1['attention_mask'][:max_length].to(device)
    output1 = model_1(input_ids=token, attention_mask=mask).logits
    
    test_text = preprocessing_text(test_text)
    tokenizer_output2 = tokenizer.encode_plus(test_text,
                                          max_length=max_length,
                                          padding="max_length",
                                          return_tensors="pt"
                                         )
    token = tokenizer_output2['input_ids'][:max_length].to(device)
    mask = tokenizer_output2['attention_mask'][:max_length].to(device)
    output2 = model_2(input_ids=token, attention_mask=mask).logits
    
    output = (output1 + output2)/2

In [17]:
int(torch.argmax(output).cpu())

0

In [92]:
def predict(text):
    with torch.no_grad():
        tokenizer_output = tokenizer.encode_plus(text,
                                              max_length=max_length,
                                              padding="max_length",
                                              return_tensors="pt"
                                             )
        token = tokenizer_output['input_ids'][0][:max_length].view(1,max_length).to(device)
        mask = tokenizer_output['attention_mask'][0][:max_length].view(1,max_length).to(device)
        output1 = model_1(input_ids=token, attention_mask=mask).logits

        text = preprocessing_text(text)
        token = tokenizer_output['input_ids'][0][:max_length].view(1,max_length).to(device)
        mask = tokenizer_output['attention_mask'][0][:max_length].view(1,max_length).to(device)
        output2 = model_2(input_ids=token, attention_mask=mask).logits
    
    output = 0.4*output1 + 0.6*output2
    return int(torch.argmax(output).cpu())

In [93]:
submission['Категория'] = submission['text'].apply(predict)

/tmp/ipykernel_4979/2734527468.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Категория'] = submission['text'].apply(predict)


In [94]:
submission.head()

,id,text,Категория
0,843,<p>Здравствуйте. На улице Мира &nbsp;было заме...,0
1,1422,<p>Уже вторую неделю не горит уличное освещени...,0
2,2782,Не работает освещение во дворе дома 11а по Эне...,0
3,2704,После покоса сорной растительности на газоне м...,16
4,1,<p>Прошу принять меры к водителю маршрута 263:...,8


In [95]:
pred_sub = submission[['id', 'Категория']]
pred_sub.to_csv('submission4.csv', index=False)